In [ ]:
import numpy as np

def northwest_corner(costs, supply, demand):
    """Gera solução inicial pelo método do canto noroeste."""
    supply = supply.copy()
    demand = demand.copy()
    allocation = np.zeros_like(costs, dtype=int)

    i, j = 0, 0
    while i < 3 and j < 3:
        qty = min(supply[i], demand[j])
        allocation[i][j] = qty
        supply[i] -= qty
        demand[j] -= qty
        if supply[i] == 0 and demand[j] == 0:
            i += 1
            j += 1
        elif supply[i] == 0:
            i += 1
        else:
            j += 1
    return allocation

def calculate_potentials(allocation, costs):
    """Calcula potenciais u,v e custos reduzidos delta."""
    u = [None]*3
    v = [None]*3
    u[0] = 0
    basic = [(i,j) for i in range(3) for j in range(3) if allocation[i][j] > 0]

    changed = True
    while changed:
        changed = False
        for i,j in basic:
            if u[i] is not None and v[j] is None:
                v[j] = costs[i][j] - u[i]
                changed = True
            elif v[j] is not None and u[i] is None:
                u[i] = costs[i][j] - v[j]
                changed = True

    delta = np.zeros_like(costs, dtype=int)
    for i in range(3):
        for j in range(3):
            if (i,j) not in basic:
                delta[i][j] = costs[i][j] - (u[i] + v[j])
    return u, v, delta

def find_loop(allocation, start):
    """Encontra loop fechado em 3x3 para célula inicial start (i,j)."""
    i0,j0 = start
    basic = [(i,j) for i in range(3) for j in range(3) if allocation[i][j] > 0]
    basic.append((i0,j0))

    # Para 3x3, tentamos percorrer linhas e colunas alternadas
    # Geramos todas combinações possíveis simples (como L)
    for i1,j1 in basic:
        if i1 != i0 and j1 != j0 and (i1,j0) in basic and (i0,j1) in basic:
            return [(i0,j0),(i0,j1),(i1,j1),(i1,j0)]
    return None

def modi_3x3(costs, supply, demand):
    allocation = northwest_corner(costs, supply, demand)

    while True:
        u, v, delta = calculate_potentials(allocation, costs)
        if np.all(delta >= 0):
            break  # ótimo
        # seleciona célula com delta mínimo
        i,j = np.unravel_index(np.argmin(delta), delta.shape)
        loop = find_loop(allocation, (i,j))
        if loop is None:
            raise Exception("Não foi possível encontrar loop para melhoria")

        plus = loop[0::2]
        minus = loop[1::2]
        theta = min([allocation[x,y] for x,y in minus])

        for x,y in plus:
            allocation[x][y] += theta
        for x,y in minus:
            allocation[x][y] -= theta

    total_cost = np.sum(allocation * costs)
    u, v, delta = calculate_potentials(allocation, costs)
    return allocation, total_cost, u, v, delta

In [ ]:
# ---------- Exemplo ----------
costs = np.array([
    [4, 6, 8],
    [2, 4, 5],
    [3, 1, 7]
])
supply = [80, 100, 120]
demand = [90, 70, 140]

alloc, cost, u, v, delta = modi_3x3(costs, supply, demand)

print("Matriz Ótima:\n", alloc)
print("Custo Total Ótimo:", cost)
print("Potenciais u:", u)
print("Potenciais v:", v)
print("Custos Reduzidos (Delta):\n", delta)


Matriz Ótima:
 [[ 80   0   0]
 [  0   0 100]
 [ 10  70  40]]
Custo Total Ótimo: 1200
Potenciais u: [0, np.int64(-3), np.int64(-1)]
Potenciais v: [np.int64(4), np.int64(2), np.int64(8)]
Custos Reduzidos (Delta):
 [[0 4 0]
 [1 5 0]
 [0 0 0]]
